In [4]:
# Visualization in t-SNE or PCA

import os
import numpy as np
import plotly.express as px
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [5]:
def viz_dim_red(data_dir, method=None, save_fig=False, filename=None):
    # data_dir = r"C:\Users\ajayt\OneDrive\Desktop\Main P\other\our_method\medium_100_npy"
    # train_dir = os.path.join(data_dir, 'train')

    base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

    def load_data(directory, sample_size=100): # data dir struct as per MAIN README, so as searching
        filenames = []
        labels = []
        features = []
        
        dirs = sorted(os.listdir(directory))
        one_hots = np.eye(len(dirs))
        
        for i, folder in enumerate(dirs):
            folder_path = os.path.join(directory, folder)
            files = os.listdir(folder_path)[:sample_size]
            for file in files:
                file_path = os.path.join(folder_path, file)
                data = np.load(file_path, mmap_mode='r')
                data = np.float32(data)
                data = np.mean(data, axis=0)
                data = np.expand_dims(data, axis=0)

                feature = model.predict(data)
                features.append(feature[0])
                filenames.append(file_path)
                labels.append(one_hots[i])
        
        return np.array(features), np.array(labels), dirs, filenames

    features, labels, class_labels, filenames = load_data(data_dir)

    # Dimensionality reduction option
    if method == 'PCA':
        reducer = PCA(n_components=2)
    elif method == 't-SNE':
        reducer = TSNE(n_components=2, perplexity=30, learning_rate=200)

    reduced_features = reducer.fit_transform(features)

    hover_filenames = [os.path.basename(f) for f in filenames]

    fig = px.scatter(
        x=reduced_features[:, 0],
        y=reduced_features[:, 1],
        color=np.argmax(labels, axis=1).astype(str),
        hover_name=hover_filenames,
        title=f'Interactive {method} Visualization of RWF-2000 Features'
    )

    fig.update_layout(
        xaxis_title='Component 1',
        yaxis_title='Component 2',
        legend_title='Label',
        hoverlabel=dict(bgcolor="white", font_size=16, font_family="Rockwell")
    )

    fig.show()

    if save_fig:
        save_path = r"C:\Users\ajayt\OneDrive\Desktop\Main P"
        fig.write_html(save_path + f"\{filename}.html") # comment if you don't want saved file of viz

In [6]:
train_path = r"C:\Users\ajayt\OneDrive\Desktop\Main P\other\HockeyFloat\train"
val_path=r"C:\Users\ajayt\OneDrive\Desktop\Main P\other\HockeyFloat\val"

In [7]:
# Train dir with t-SNE 
viz_dim_red(train_path, method='t-SNE', save_fig=True, filename='train_TSNE')

1/1 [==============================] - 0s 22ms/step


In [8]:
# Train dir with PCA
viz_dim_red(train_path, method='PCA', save_fig=True, filename='train_PCA')

1/1 [==============================] - 0s 19ms/step


In [9]:
# Validation dir with t-SNE
viz_dim_red(val_path, method='t-SNE', save_fig=True, filename='val_TSNE')

1/1 [==============================] - 0s 34ms/step


In [10]:
# Validation dir with PCA 
viz_dim_red(val_path, method='PCA', save_fig=True, filename='val_PCA')

1/1 [==============================] - 0s 32ms/step
